# Explore here

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, load_img
from sklearn.metrics import precision_score, recall_score
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import random
import os
from PIL import Image

2024-01-06 12:59:11.357369: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 12:59:13.346149: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 12:59:13.346246: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 12:59:13.648823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 12:59:14.300279: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 12:59:14.301476: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
dataset_dir = '/workspaces/LungsXray-FP/data/raw/MULTIPLE'

In [3]:
image_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 5266 images belonging to 4 classes.
Found 1315 images belonging to 4 classes.


In [4]:
num_classes = len(train_generator.class_indices)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Clasificación multiclase, usar 'softmax'
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
epochs = 10

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10


2024-01-05 09:06:16.694133: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 89718784 exceeds 10% of free system memory.
2024-01-05 09:06:17.262551: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42467328 exceeds 10% of free system memory.
2024-01-05 09:06:17.944567: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 42467328 exceeds 10% of free system memory.
2024-01-05 09:06:18.024895: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 29859840 exceeds 10% of free system memory.
2024-01-05 09:06:18.178137: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.


165/165 [==============================] - 240s 1s/step - loss: 0.7900 - accuracy: 0.6910 - val_loss: 0.4362 - val_accuracy: 0.8373
Epoch 2/10
165/165 [==============================] - 226s 1s/step - loss: 0.3815 - accuracy: 0.8576 - val_loss: 0.3396 - val_accuracy: 0.8707
Epoch 3/10
165/165 [==============================] - 257s 2s/step - loss: 0.3432 - accuracy: 0.8688 - val_loss: 0.3778 - val_accuracy: 0.8487
Epoch 4/10
165/165 [==============================] - 228s 1s/step - loss: 0.3016 - accuracy: 0.8904 - val_loss: 0.3060 - val_accuracy: 0.8966
Epoch 5/10
165/165 [==============================] - 235s 1s/step - loss: 0.2591 - accuracy: 0.9041 - val_loss: 0.2863 - val_accuracy: 0.9027
Epoch 6/10
165/165 [==============================] - 228s 1s/step - loss: 0.2323 - accuracy: 0.9144 - val_loss: 0.2612 - val_accuracy: 0.8996
Epoch 7/10
165/165 [==============================] - 223s 1s/step - loss: 0.2127 - accuracy: 0.9229 - val_loss: 0.2187 - val_accuracy: 0.9224
Epoch 8/10

In [6]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nExactitud en el conjunto de validación:', test_acc)


42/42 - 32s - loss: 0.2506 - accuracy: 0.9080 - 32s/epoch - 752ms/step

Exactitud en el conjunto de validación: 0.9079847931861877


In [7]:
model.save('modelo_softmax.keras')

In [3]:
model = load_model('modelo_softmax.keras')  # Ajusta el nombre de tu modelo

# Preprocesamiento de datos para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Ajusta si tu modelo espera etiquetas categóricas
    shuffle=False
)

# Obtener las predicciones del modelo para el conjunto de prueba
predictions = model.predict(test_generator)

# Convertir las predicciones a clases (si es necesario)
predicted_classes = np.argmax(predictions, axis=1)

# Obtener las etiquetas verdaderas
true_classes = test_generator.classes

# Calcular la precisión y el recall por clase
classification_rep = classification_report(true_classes, predicted_classes)
accuracy = accuracy_score(true_classes, predicted_classes)
conf_matrix = confusion_matrix(true_classes, predicted_classes)

print("Informe de clasificación:")
print(classification_rep)

print("Exactitud general:")
print(accuracy)

print("Matriz de confusión:")
print(conf_matrix)

2024-01-06 12:59:26.555234: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-06 12:59:26.582525: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-06 12:59:26.587475: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-06 12:59:26.656792: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-06 12:59:26.667627: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.


Found 6581 images belonging to 4 classes.
206/206 [==============================] - 139s 673ms/step
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      1589
           1       0.93      0.94      0.93      1341
           2       0.96      0.96      0.96      2616
           3       0.99      0.84      0.91      1035

    accuracy                           0.94      6581
   macro avg       0.95      0.93      0.94      6581
weighted avg       0.95      0.94      0.94      6581

Exactitud general:
0.9436255888162893
Matriz de confusión:
[[1559    4   15   11]
 [  17 1257   67    0]
 [  17   77 2520    2]
 [ 138   12   11  874]]


In [9]:
# Lista de carpetas en el directorio
folders = os.listdir(dataset_dir)

# Modelo preentrenado
model = load_model('modelo_softmax_optimizado.keras')

# Número de imágenes a seleccionar por carpeta
num_images_por_carpeta = 25

# Crear un diccionario que mapea índices a nombres de carpetas
index_to_folder = {i: folder for i, folder in enumerate(folders)}

# Iterar sobre cada carpeta
for folder in folders:
    folder_path = os.path.join(dataset_dir, folder)
    
    # Lista de archivos en la carpeta
    files = os.listdir(folder_path)
    
    # Seleccionar aleatoriamente 25 imágenes
    selected_files = random.sample(files, num_images_por_carpeta)
    
    # Iterar sobre cada imagen seleccionada
    for file in selected_files:
        file_path = os.path.join(folder_path, file)
        
        # Cargar la imagen y preprocesarla
        img = load_img(file_path, target_size=(150, 150))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0  # Normalizar
        
        # Hacer la predicción con el modelo
        prediction = model.predict(img_array)
        
        # Obtener la clase predicha
        predicted_class = np.argmax(prediction)
        
        # Obtener el nombre de la carpeta asociada a la clase predicha
        predicted_folder = index_to_folder[predicted_class]
        
        # Mostrar resultados
        print(f"Imagen: {file}, Carpeta Real: {folder}, Carpeta Predicha: {predicted_folder}")














1/1 [==============================] - 0s 354ms/step
Imagen: COVID-19 (467).jpg, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 19ms/step
Imagen: COVID-19 (589).jpg, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 24ms/step
Imagen: 103.jpeg, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 22ms/step
Imagen: 508.jpeg, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 25ms/step
Imagen: 014.jpeg, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 21ms/step
Imagen: COVID19(243).jpg, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 19ms/step
Imagen: 041.png, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 24ms/step
Imagen: COVID-19 (810).jpg, Carpeta Real: COVID, Carpeta Predicha: COVID
1/1 [==============================] - 0s 